In [1]:
using DataFrames
using CSV
using LinearAlgebra
include("flux_ventilation.jl")
include("building_network_model.jl")
include("flux_ventilation.jl")

(-6.409261176238541, 0.0, 6.409261176238541, 1.4614107443030637, 0.0, 1.4614107443030637)

# 多数室換気計算プログラム

本プログラムでは多数室における換気計算方法について示す。  
なお、各計算においては主に以下のプログラムを参照のこと
- building_network_model.ipynb：換気回路網に関するモデル化方法を表す
- opening.ipynb：開口の条件を表す
- flux_ventilation.ipynb：開口における流量の計算式を表す  

なお、ここではBNM(Building Network Model)が構築されていることを前提とした関数化を行う。  
以下、多数室換気計算における主な計算の流れを示す。

In [2]:
# 読み込み例
test_BNM = create_BNM_model(
    file_name_rooms     = "../input_data/building_network_model/room_condition_for_check.csv", 
    file_name_walls     = "../input_data/building_network_model/wall_condition_for_check.csv", 
    file_name_openings  = "../input_data/building_network_model/opening_condition_for_check.csv",
    file_name_climate   = "../input_data/climate_data/sample_climate_data.csv")

BNM(Room[Room(1, "outdoor", Air(0, "climate", 0.0, 0.0, 0.0, 0.0, 276.84999999999997, 0.77, 613.2898775721085, 0.0037877055546582487, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, "OFF"), Room(2, "room1", Air(0, "room1", 0.0, 0.0, 0.0, 125.0, 283.15, 0.6, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, "OFF"), Room(3, "room2", Air(0, "room2", 0.0, 0.0, 0.0, 125.0, 283.15, 0.6, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.025, 0.0, 0.0, 0.0, "OFF")], Wall[Wall(1, "ground", 1, 2, 0.0, 0.2, 100.0, 0.0, 0.0, Cell[Cell([1, 1, 1], [0.0, 0.0, 0.0], 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 283.15, -66807.7360789275, 0.0, 0.0, 0.0, 101325.0, "sandy_clay_loam", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]), Cell([2, 1, 1], [0.0, 0.0, 0.0], 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 283.15, -66807.7360789275, 0.0, 0.0, 0.0, 101325.0, "sand

## 手順１：仮定床面圧力Pの初期値設定

### 関数名（Fortran時）：SUBROUTINE PCAL

仮定床面圧力[Pa]を求めるための関数（処理）。  
各室における床面圧力Pを適当に配置するための関数。  
forループ内では余剰や乗算、割り算などを活用することで適当な（計算ができる程度の大きさを持つ）床面圧力を設定している。

- （仮定）床面圧力[Pa]
- RM：室の数
- NNおよびX：部屋の最大数（適当でよい）
- XMOD, XMOD1：床面圧力を適当に決めるための変数  

※2021/07/27 IRを削除

In [3]:
function set_pf_assumption( rooms::Array{Room, 1} )
    NN   = 50
    XMOD = 10.0e+6
    XMOD1= XMOD + 1.0
    X    = NN
    for i = eachindex(rooms)
        rooms[i].pf = mod( X, XMOD1 ) / XMOD
        X    = 15.0 * X
        if X > 1.0e+20 
            X = X / 1.0e+14
        end
    end
    rooms[1].pf = 0.0
end

set_pf_assumption( BNM::BNM ) = set_pf_assumption( BNM.rooms )

set_pf_assumption (generic function with 2 methods)

In [4]:
set_pf_assumption( test_BNM )

0.0

## 手順2：圧力差の計算

#### 関数名（Fortran時）：SUBROUTINE PSCAL

上流から下流方向への流れを正とする。  
すなわち、圧力差は開口上流圧力 - 開口下流圧力を正とする。

### 2.1 起圧力の計算  
床面の高さの違いは位置圧に影響を及ぼす。  
以下では高さの違いに伴う各床面の圧力について計算を行う。

※床面高さの差が無いとした場合の静圧　=　仮定床面圧力  

In [5]:
# 起圧力の計算
function cal_Pressuremotive_force( opening::Opening )
    if opening.room_IP.Hight >= opening.room_IM.Hight
        rhoa = 353.25 / temp(opening.room_IM)
    else
        rhoa = 353.25 / temp(opening.room_IP)
    end
    return - rhoa * 9.80665 * ( opening.room_IP.Hight - opening.room_IM.Hight )
end

cal_Pressuremotive_force (generic function with 1 method)

In [6]:
cal_Pressuremotive_force( test_BNM.openings[1] )

0.250258198482933

### 2.2 風圧力の計算

$$
    p_w = C p_v = C \Bigl( \frac{\rho}{2}V^2 \Bigl)
$$  

In [7]:
# 風圧力の計算
function cal_wind_pressure_force( opening::Opening, climate::Climate )
    Pw_IP = cal_wind_pressure_force( C = WC_IP(opening), rho = 353.25 / temp(opening.room_IP), v = WS(climate) )
    Pw_IM = cal_wind_pressure_force( C = WC_IM(opening), rho = 353.25 / temp(opening.room_IM), v = WS(climate) )
    return Pw_IP - Pw_IM
end

cal_wind_pressure_force (generic function with 3 methods)

In [8]:
# 風圧係数の計算
function set_wind_pressure_coefficient( BNM::BNM )
    for i = 1 : length(BNM.openings)
        # 風圧係数の初期化
        setfield!(BNM.openings[i], :WC_IP, 0.0)
        setfield!(BNM.openings[i], :WC_IM, 0.0)
        if BNM.IC_openings[1,i] == 1   # インシデンス行列よりIP側がClimateである場合を判別
            WC  = :WC_IP
            DIR = :DIR_IP
        elseif BNM.IC_openings[1,i] == -1   # インシデンス行列よりIM側がClimateである場合を判別
            WC  = :WC_IM
            DIR = :DIR_IM
        else
            continue # 以下の動作をスキップ
        end
        # 風圧係数のセット
        setfield!(BNM.openings[i], WC, 
                    cal_wind_pressure_coefficient(direction_JP[WD(BNM.climate)] - direction[getfield(BNM.openings[i], DIR)]))
    end
end

set_wind_pressure_coefficient (generic function with 1 method)

### 2.3 圧力差の計算

In [9]:
# 圧力差の計算
function set_dP_openings( BNM::BNM )
    # 風圧係数のセット
    set_wind_pressure_coefficient( BNM )
    for i = 1 : length( BNM.openings )
        dPf = pf(BNM.rooms[IP(BNM.openings[i])]) - pf(BNM.rooms[IM(BNM.openings[i])])
        dPs = cal_Pressuremotive_force( BNM.openings[i] )
        dPw = cal_wind_pressure_force( BNM.openings[i], BNM.climate )
        setfield!(BNM.openings[i], :dP, dPf + dPs + dPw)
    end
end

function reset_dPf_openings( BNM::BNM )
    for i = 1 : length( BNM.openings )
        dPf = pf(BNM.rooms[IP(BNM.openings[i])]) - pf(BNM.rooms[IM(BNM.openings[i])])
        dPs = cal_Pressuremotive_force( BNM.openings[i] )
        dPw = cal_wind_pressure_force( BNM.openings[i], BNM.climate )
        setfield!(BNM.openings[i], :dP, dPf + dPs + dPw)
    end
end

reset_dPf_openings (generic function with 1 method)

In [10]:
set_dP_openings( test_BNM )

## 手順３：流量計算

### 3.1 開口流量の計算

#### 関数名（Fortran時）：SUBROUTINE FLCAL  
#### 2点間の温度・圧力差から空気の流量を計算する
※module_calc_ventilationを用いて計算を行う。  
⇒　計算の詳細はmodule_calc_ventilationを参照のこと。

※下記の2つは入力条件として存在しないので別途計算が必要  
QV：縦方向開口1Pa時[m<sup>3</sup>/s]を長辺長さで割ったもの[m<sup>2</sup>/s]  
QH：横方向開口の1Pa時[m<sup>3</sup>/s]を長辺長さで割ったもの[m<sup>2</sup>/s]

In [11]:
function cal_flux_ventilation( opening::Opening )
    RAW_IP  = 353.25 / temp(opening.room_IP)
    RAW_IM  = 353.25 / temp(opening.room_IM)
    DRAW    = RAW_IP - RAW_IM
    return cal_flux_ventilation(    ION = opening.ION, Type = opening.Type, 
                                    DP  = opening.dP, 
                                    DRAW= DRAW, 
                                    A   = opening.A,    B  = opening.B,     S = opening.S, 
                                    HU  = opening.HU,   HD = opening.HD, 
                                    M   = opening.M,    MM = opening.MM, 
                                    RAW_IP = RAW_IP,    RAW_IM = RAW_IM, 
                                    QV  = opening.QV,   QH  = opening.QH )
end

function set_flux_ventilation( BNM::BNM )
    # 各開口における流量の計算
    for i =  1 :length(BNM.openings)
        flux = BNM.openings[i].flux
        flux["W"], flux["WU"], flux["WD"], flux["DW"], flux["DWU"], flux["DWD"] = cal_flux_ventilation( BNM.openings[i] )
    end
end

set_flux_ventilation (generic function with 1 method)

In [12]:
for i = 1 : length(test_BNM.openings)
    test_BNM.openings[i].Type = "gap"
end

In [13]:
set_flux_ventilation( test_BNM )

In [14]:
for i = 1 : length(test_BNM.openings)
    #println(test_BNM.openings[i].dP)
    println(test_BNM.openings[i].flux["W"])
    #println(cal_flux_ventilation( test_BNM.openings[i] ))
end

-60.66205415788184
91.73535712188394
21.78397436175036
-60.08536548582525
-69.69254359807347


### 3.2 正味の流量を各室に設定

#### 関数名（Fortran時）：SUBROUTINE DWCAL  
#### 室に流入する正味の流量

- V_SM： 同様枝数
- DWW：室正味流量
- W：開口の正味流量
- IC：インシデンス行列

In [15]:
# 室への正味流量
function set_DWW_of_rooms( BNM::BNM )
    for i = 1 : length(BNM.rooms)
        BNM.rooms[i].DWW = 0.0
        for j = 1 : length(BNM.openings)
            BNM.rooms[i].DWW = BNM.rooms[i].DWW + BNM.IC_openings[i,j] * BNM.openings[j].flux["W"]
        end
    end
end

set_DWW_of_rooms (generic function with 1 method)

In [16]:
set_DWW_of_rooms( test_BNM )

## 4 圧力仮定法による流量誤差の計算

室内における質量収支は行列を用いて以下のように表される。  

$$
    \frac{\partial \rho V}{\partial t} = I{w} - {W}
$$

- $\rho$：室の密度
- $V$：室の体積
- $t$：時間
- $I$：インシデンス行列
- $w$：開口部の正味流量
- $W$：室内における質量発生量（CO2など）  

なお、エネルギー収支が無条件に満足され、温度変化が無い場合には収支は0となる。  
圧力仮定法とはこのように節点での質量収支を満足するよう$w$あるいは各室の圧力を修正する方法である。

### 4.1 流量誤差の近似方法

全室に関する流量収支の誤差を許容するよう全室の床面圧力を同時に変化させる逐次近似計算法について示す。正確値に対し偏差$\Delta$を含んだ近似値に対して、次式が成立する。

(1) 開口部圧力差pと室圧力Pとの関係
$$
    (p + \Delta p) + p_{\rho} = I' (P+\Delta P)
$$

- $P$：正確な値の室圧力
- $p$：正確な値の枝圧力差
- $p_{\rho}$：ファンなど動力源により枝に加わる圧力

(2) 開口部の正味流量wと圧力差pとの関係
$$
    w + \Delta w = f(p, \rho_i, \rho_j, \alpha, b, h)
$$

(3) 室における濃度収支式
$$
    \Delta W =  I ( w + \Delta w ) - W 
$$


近似値が正確値に近いとき、上述の式から正確値を差し引くと偏差の関係式として以下の関係が得られる。  

(1') 開口部圧力差pと室圧力Pとの関係
$$
    \Delta p = I' \Delta P
$$

(2') 開口部の正味流量wと圧力差pとの関係
$$
    \Delta w = \frac{\partial f}{\partial p} \Delta p
$$

(3') 室における濃度収支式
$$
    \Delta W =  I \Delta w 
$$

なお、(2)式の開口部の正味流量は圧力差に対してTaylor展開し、2次以上の項を無視することで求めた。  

従って、式(3')に式(2')および式(1')を代入することで以下の関係式が得られる。  

$$
    \Delta W = \Biggl ( I \Biggl [ \frac{\partial f}{\partial p} \Biggl ] I' \Biggl ) \Delta P
$$


In [17]:
# 行列を用いた計算
function cal_IDD( BNM::BNM )
    dfdp = [ BNM.openings[i].flux["DW"] for i = 1 : length(BNM.openings)  ]
    return BNM.IC_openings * Diagonal(dfdp) * BNM.IC_openings'
end    

cal_IDD (generic function with 1 method)

In [18]:
# 従来の計算方法（若干計算速度が速い）
function cal_IDD_original( BNM::BNM )
    
    IDD = zeros(Float64, length(BNM.rooms), length(BNM.rooms))

    for i = 1 : length(BNM.openings)
        IDD[ BNM.openings[i].IP, BNM.openings[i].IP ] = IDD[BNM.openings[i].IP, BNM.openings[i].IP] + BNM.openings[i].flux["DW"] #([I][∂f/∂p][I'])
        IDD[ BNM.openings[i].IP, BNM.openings[i].IM ] = IDD[BNM.openings[i].IP, BNM.openings[i].IM] - BNM.openings[i].flux["DW"]
        IDD[ BNM.openings[i].IM, BNM.openings[i].IP ] = IDD[BNM.openings[i].IP, BNM.openings[i].IM]
        IDD[ BNM.openings[i].IM, BNM.openings[i].IM ] = IDD[BNM.openings[i].IM, BNM.openings[i].IM] + BNM.openings[i].flux["DW"]
    end

    return IDD
end    

cal_IDD_original (generic function with 1 method)

In [19]:
cal_IDD( test_BNM )

3×3 Matrix{Float64}:
  341.915  -162.196  -179.719
 -162.196   363.489  -201.292
 -179.719  -201.292   381.011

In [20]:
cal_IDD_original( test_BNM )

3×3 Matrix{Float64}:
  341.915  -162.196  -179.719
 -162.196   363.489  -201.292
 -179.719  -201.292   381.011

あるいは、

$$
    \Delta P =  \Biggl (  I \Biggl [ \frac{\partial f}{\partial p} \Biggl ] I' \Biggl )^{-1} \Delta W
$$

In [21]:
# Moore-Penrose逆行列（要検討）
function cal_inverse_IDD(BNM::BNM)
    return pinv( cal_IDD( BNM ) )
    #return inv( cal_IDD( BNM ) )
end

cal_inverse_IDD (generic function with 1 method)

### 4.2 Newton-Raphson法による収束計算

Newton-Raphson法は、方程式の解を数値計算により解くための反復法による求根アルゴリズムの一つである。  
室における質量収支が満足される場合、


<img src="picture/Newton_iteration.png" width="50%">

従って、$P - c \Delta P$により節点圧力の補正を行っていくこととなる。  


In [22]:
function cal_modified_pf_by_Newton_Raphson_method( BNM::BNM )

    # 補正緩和係数 <= 1.0
    ck = 1.0

    IDD = cal_inverse_IDD( BNM )
    IC  = BNM.IC_openings
    #dW  = [ BNM.openings[i].flux["DW"] for i = 1 : length( BNM.openings ) ]
    #dW  = [ BNM.openings[i].flux["W"] for i = 1 : length( BNM.openings ) ]
    dW  = [ BNM.rooms[i].DWW for i = 1 : length( BNM.rooms ) ]
    dP  = IDD * dW
    #dP  = IDD * IC * dW
    #dP  = IDD * IC
    #dP  = IDD
    
    # 修正圧力計算
    #for i = 1 : length( BNM.rooms ) #外気の床面圧力も補正しているが良いのか？
    for i = 2 : length( BNM.rooms ) #外気の床面圧力も補正しているが良いのか？
        # 各部屋の床面圧力を補正する
        BNM.rooms[i].pf = BNM.rooms[i].pf - ck * dP[i]
    end

    # 開口の圧力差の再計算
    for i = 1 : length( BNM.openings )
        dPf = pf(BNM.rooms[IP(BNM.openings[i])]) - pf(BNM.rooms[IM(BNM.openings[i])])
        setfield!(BNM.openings[i], :dP, BNM.openings[i].dP + dPf )
    end
    
end

cal_modified_pf_by_Newton_Raphson_method (generic function with 1 method)

In [23]:
# 逆行列の計算　⇒　inv(x)に変更で良い？
function cal_modified_pf_by_Newton_Raphson_method_original( BNM::BNM );

    
    # 補正緩和係数 <= 1.0
    ck = 0.1

    IDD = cal_inverse_IDD( BNM )

    DDP = zeros(length(BNM.rooms))

    for IK = 2 : length(BNM.rooms)
        PIVOT = IDD[ IK, IK ]
        for IJ = IK : length(BNM.rooms)
            IDD[IK,IJ] = IDD[IK,IJ] / PIVOT
        end

        for I = 2 : length(BNM.rooms)
            if I == IK
                continue # I == IKならばループの最初に戻る。
            end
            
            CIK = IDD[I,IK] # CIKこの式はcontinueの前では...？
            
            for IJ = IK : length(BNM.rooms)
                IDD[I,IJ] = IDD[I,IJ] - CIK * IDD[IK,IJ]
            end
        end
    end

    for i = 2 : length(BNM.rooms)
        DDP[i] = IDD[ i, length(BNM.rooms) ]
    end

    # 修正圧力計算
    for i = 2 : length( BNM.rooms )
        # 各部屋の床面圧力を補正する
        BNM.rooms[i].pf = BNM.rooms[i].pf - ck * DDP[i]
    end

    # 開口の圧力差の再計算
    for i = 1 : length( BNM.openings )
        dPf = pf(BNM.rooms[IP(BNM.openings[i])]) - pf(BNM.rooms[IM(BNM.openings[i])])
        setfield!(BNM.openings[i], :dP, BNM.openings[i].dP + dPf )
    end
    

end

cal_modified_pf_by_Newton_Raphson_method_original (generic function with 1 method)

In [24]:
cal_modified_pf_by_Newton_Raphson_method( test_BNM )

In [25]:
cal_modified_pf_by_Newton_Raphson_method_original( test_BNM )

In [26]:
for i = 1 : length( test_BNM.rooms ) 
    println(test_BNM.rooms[i].pf)
end

0.0
0.0034318428810537842
-0.2913410107115428


## 5. 反復計算

In [27]:
maximum([1,24,4])

24

In [28]:
maximum([ abs(test_BNM.openings[i].flux["DW"]) for i = 1:length(test_BNM.openings)] )

201.29246827140744

In [29]:
function cal_multi_ventilation( BNM::BNM )

    step        = 0
    step_limit  = 500
    count       = 0
    ERRLIM      = 1.0e-8

    # 1. 仮定床面圧力の設定
    set_pf_assumption( BNM )

    # 2. 初期の圧力差の計算
    set_dP_openings( BNM )

    # 収束計算
    while true
        # 3. 流量計算
        set_flux_ventilation( BNM )
        set_DWW_of_rooms( BNM )

        # 流量誤差の計算
        DWWMAX = maximum([ abs(BNM.rooms[i].DWW) for i = 1:length(BNM.rooms)] )

        # 収束の判別
        if DWWMAX * 3600.0 > ERRLIM
            #println("step = ", step, " ERRLIM = ", ERRLIM)
            
            # 4. 修正圧力計算
            cal_modified_pf_by_Newton_Raphson_method( BNM )
            #cal_modified_pf_by_Newton_Raphson_method_original( BNM )
            
            # 収束条件の緩和条件
            step = step + 1

            if step > step_limit
                step    = 0
                count   = count + 1
                ERRLIM  = ERRLIM * count
                println(" pf1 = ", test_BNM.rooms[1].pf, " pf2 = ", test_BNM.rooms[2].pf, " pf3 = ", test_BNM.rooms[3].pf)
                println(" ERRLIM = ", ERRLIM)
                println(" DWWMX = ", DWWMAX)
            end
        else
            break
        end
    end
end

cal_multi_ventilation (generic function with 1 method)

In [30]:
cal_multi_ventilation( test_BNM )

 pf1 = 0.0 pf2 = -0.23629661587016193 pf3 = 0.7569881450601599
 ERRLIM = 1.0e-8
 DWWMX = 123.80383015996225
 pf1 = 0.0 pf2 = -0.6765268409065105 pf3 = 0.35262726342706124
 ERRLIM = 2.0e-8
 DWWMX = 247.35285490040405
 pf1 = 0.0 pf2 = -0.07729088984322552 pf3 = -0.2648508982808928
 ERRLIM = 6.000000000000001e-8
 DWWMX = 406.6861902769444
 pf1 = 0.0 pf2 = 1.4884191490306606 pf3 = -0.9139644642722149
 ERRLIM = 2.4000000000000003e-7
 DWWMX = 278.5532249989407
 pf1 = 0.0 pf2 = 1.41022426294097 pf3 = -1.7592165498351162
 ERRLIM = 1.2000000000000002e-6
 DWWMX = 421.82391146420474
 pf1 = 0.0 pf2 = -1.6788182768318525 pf3 = 0.16083809466198473
 ERRLIM = 7.2000000000000005e-6
 DWWMX = 388.3787419500468
 pf1 = 0.0 pf2 = 0.4811802734623283 pf3 = -0.6237295687233329
 ERRLIM = 5.0400000000000005e-5
 DWWMX = 657.335985471315
 pf1 = 0.0 pf2 = -0.5191838993316682 pf3 = 1.2877308833560077
 ERRLIM = 0.00040320000000000004
 DWWMX = 727.5854588462312
 pf1 = 0.0 pf2 = -2.0883587257496683 pf3 = -1.42289435906

In [31]:
for i = 1 : length(test_BNM.rooms )
    println( test_BNM.rooms[i].DWW )
end

628.274536587294
78.31691139635714
-706.5914479836513


In [32]:
for i = 1 : length(test_BNM.openings )
    println( test_BNM.openings[i].flux["W"] )
end

23.784108292061013
127.75156127366122
229.85258096207937
240.0366873456793
236.70217967589258
